In [2]:
# requests 라이브러리 설치여부 확인
!pip show requests

Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: aext-assistant-server, anaconda-auth, anaconda-catalogs, anaconda-client, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract, webdriver-manager


In [3]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


### 2-1. Nate 뉴스기사 제목 스크래핑하기

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

def scrape_nate_news(section_code='n0300'):  # 기본은 경제
    url = f'https://news.nate.com/recent?mid={section_code}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    articles = soup.select('div.postSubjectContent > div.mduSubjectList')  # 각 뉴스 블록

    for article in articles[:5]:  # 앞 5개만 출력
        a_tag = article.select_one('a')
        if not a_tag:
            continue

        # 제목
        title_tag = a_tag.select_one('h2.tit')
        title = title_tag.get_text(strip=True) if title_tag else "[No Title]"

        # 링크
        href = a_tag['href']
        link = urljoin('https://news.nate.com', href)

        # 이미지 (없을 수도 있음)
        img_tag = a_tag.find('img')
        if img_tag and img_tag.get('src'):
            img_url = urljoin("https:", img_tag['src'])
            display(Image(url=img_url))
        else:
            print("[No Image]")

        print("제목:", title)
        print("링크:", link)
        print("-" * 60)

In [5]:
# 최신뉴스: n0100
# 정치: n0201
# 경제: n0300
# 사회: n0400
# 세계: n0502
# IT/과학: n0600

scrape_nate_news('n0300')  # 경제 뉴스 출력

제목: 'HBM 훈풍' SK하이닉스…2분기 삼성전자보다 2배 벌었다(종합)
링크: https://news.nate.com/view/20250724n38970?mid=n0300
------------------------------------------------------------


제목: "수출계약 유보되기도"…美 관세 불확실성에 기업심리 부진
링크: https://news.nate.com/view/20250724n02236?mid=n0300
------------------------------------------------------------


제목: 강남 재건축 '스카이브리지 전성시대' 저무나
링크: https://news.nate.com/view/20250724n34744?mid=n0300
------------------------------------------------------------


제목: 전국 토지거래 줄어들었는데 강남·용산·용인 땅값은 '들썩'
링크: https://news.nate.com/view/20250724n34739?mid=n0300
------------------------------------------------------------


제목: 무역위, 일·중 열간압연 강판에 덤핑 방지 관세 부과 건의키로
링크: https://news.nate.com/view/20250724n38946?mid=n0300
------------------------------------------------------------


### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드

In [1]:
import requests
from bs4 import BeautifulSoup
import os

# 수업 때의 header 재활용
req_header = {
    'referer': '',  # 함수 안에서 URL로 채워줄 거야
    'user-agent': ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/104.0.0.0 Safari/537.36")
}

def download_one_episode(title, no, url):
    # 디렉토리 만들기
    folder_path = f'img/{title}/{no}'
    os.makedirs(folder_path, exist_ok=True)

    # 1) 웹툰 회차 페이지 가져오기
    req_header['referer'] = url
    resp = requests.get(url, headers=req_header)
    if not resp.ok:
        print(f"[Error] 페이지 요청 실패: {resp.status_code}")
        return

    # 2) BeautifulSoup으로 img 태그 추출
    soup = BeautifulSoup(resp.text, 'html.parser')
    img_tags = soup.select('div.wt_viewer img')
    print(f"총 {len(img_tags)}개의 이미지를 찾았습니다.\n")

    # 3) 각 이미지 다운로드
    for idx, img_tag in enumerate(img_tags, start=1):
        img_url = img_tag.get('src')
        if not img_url:
            continue

        # referer는 항상 회차 페이지 URL
        req_header['referer'] = url
        img_res = requests.get(img_url, headers=req_header)
        if not img_res.ok:
            print(f"[Fail] {img_url} → {img_res.status_code}")
            continue

        # 파일명: 001.jpg 같은 형식
        ext = os.path.splitext(img_url)[1] or '.jpg'
        filename = f"{idx:03}{ext}"
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'wb') as f:
            f.write(img_res.content)
        print(f"[OK] {file_path} ({len(img_res.content):,} bytes)")

# 사용 예시
download_one_episode(
    '일렉시드', 
    341, 
    'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed'
)


총 88개의 이미지를 찾았습니다.

[OK] img/일렉시드/341\001.jpg (87,143 bytes)
[OK] img/일렉시드/341\002.jpg (256,127 bytes)
[OK] img/일렉시드/341\003.jpg (184,536 bytes)
[OK] img/일렉시드/341\004.jpg (182,867 bytes)
[OK] img/일렉시드/341\005.jpg (112,615 bytes)
[OK] img/일렉시드/341\006.jpg (169,889 bytes)
[OK] img/일렉시드/341\007.jpg (157,876 bytes)
[OK] img/일렉시드/341\008.jpg (181,837 bytes)
[OK] img/일렉시드/341\009.jpg (203,632 bytes)
[OK] img/일렉시드/341\010.jpg (113,543 bytes)
[OK] img/일렉시드/341\011.jpg (166,818 bytes)
[OK] img/일렉시드/341\012.jpg (196,171 bytes)
[OK] img/일렉시드/341\013.jpg (158,408 bytes)
[OK] img/일렉시드/341\014.jpg (203,525 bytes)
[OK] img/일렉시드/341\015.jpg (81,271 bytes)
[OK] img/일렉시드/341\016.jpg (171,092 bytes)
[OK] img/일렉시드/341\017.jpg (65,626 bytes)
[OK] img/일렉시드/341\018.jpg (135,215 bytes)
[OK] img/일렉시드/341\019.jpg (96,367 bytes)
[OK] img/일렉시드/341\020.jpg (139,925 bytes)
[OK] img/일렉시드/341\021.jpg (191,828 bytes)
[OK] img/일렉시드/341\022.jpg (211,389 bytes)
[OK] img/일렉시드/341\023.jpg (174,099 bytes)
[OK] img/일렉시드/341\